# 使用RoBERTa-wwm-ext进行文本分类任务

## 1. 准备工作

### 1.1 环境准备


In [1]:
#关闭安装的输出
# %%capture
# !pip install transformers
# !pip install datasets
# !pip install evaluate
# !pip install git+https://github.com/JunnYu/WoBERT_pytorch.git


### 1.2 函数库与gpu使用

In [2]:
import os
import torch
from torch import nn
from torch.optim import AdamW,SGD,lr_scheduler
from torch.utils.data import DataLoader
from transformers import BertModel, BertTokenizer, BertConfig
from transformers import TrainingArguments, Trainer
from datasets import load_metric
from transformers.trainer_utils import EvalPrediction
import torchtext
import pandas as pd
from sklearn.preprocessing import LabelEncoder
#import evaluate


TRAIN_BATCH_SIZE = 20
VAL_BATCH_SIZE = 20
TEST_BATCH_SIZE = 20

# 使用gpu进行训练
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using {device} device")
# gpu型号
!nvidia-smi


Using cuda device
Fri Apr 21 06:09:04 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    27W / 250W |      2MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                      

## 2. 数据预处理

### 2.1 数据集导入

In [3]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, filepath):
        news_df = pd.read_csv(filepath, encoding='utf-8')
        texts = news_df.loc[:,'text'].values
        labels = news_df.loc[:,'class'].values
    
        le = LabelEncoder()
    
        self.X = texts
        self.y = le.fit_transform(labels)
    def __len__(self):
        return len(self.y)
  
    def __getitem__(self, i):
        text = self.X[i]
        label = self.y[i]
        return text, label


# train_df = pd.read_csv('/kaggle/input/thucnews-subset-dataset/pr_train.csv',encoding='utf-8')
# val_df = pd.read_csv('/kaggle/input/thucnews-subset-dataset/pr_val.csv',encoding='utf-8')
# test_df = pd.read_csv('/kaggle/input/thucnews-subset-dataset/pr_test.csv',encoding='utf-8')
# train_df.head()

train_data = Dataset('/kaggle/input/thucnews-subset-dataset/pr_train.csv')
val_data = Dataset('/kaggle/input/thucnews-subset-dataset/pr_val.csv')
test_data = Dataset('/kaggle/input/thucnews-subset-dataset/pr_test.csv')
train_data.__getitem__(0)

('马晓旭 意外 受伤 国奥 警惕 无奈 大雨 格外 青睐 殷家 军 记者 傅亚雨 沈阳 报道 来到 沈阳 国奥队 依然 摆脱 雨水 困扰 月 日 下午 国奥队 日常 训练 再度 大雨 干扰 无奈 之下 队员 慢跑 分钟 草草收场 日 上午 国奥队 奥体中心 外场 训练 阴沉沉 气象预报 显示 当天 下午 沈阳 大雨 幸好 队伍 上午 训练 干扰 下午 点当 球队 抵达 训练场 大雨 几个 小时 丝毫 停下来 抱 试一试 态度 球队 当天 下午 例行 训练 分钟 天气 转好 迹象 保护 球员 国奥队 中止 当天 训练 全队 返回 酒店 雨 训练 足球队 稀罕 奥运会 即将 全队 变得 娇贵 沈阳 一周 训练 国奥队 保证 现有 球员 不再 出现意外 伤病 情况 影响 正式 比赛 这一 阶段 控制 训练 受伤 控制 感冒 疾病 队伍 放在 位置 抵达 沈阳 后卫 冯萧霆 训练 冯萧霆 月 日 长春 患上 感冒 参加 日 塞尔维亚 热身赛 队伍 介绍 冯萧霆 发烧 症状 两天 静养 休息 感冒 恢复 训练 冯萧霆 例子 国奥队 对雨中 训练 显得 谨慎 担心 球员 受凉 引发 感冒 非战斗 减员 女足 队员 马晓旭 热身赛 受伤 导致 无缘 奥运 前科 沈阳 国奥队 格外 警惕 训练 嘱咐 队员 动作 再出 事情 一位 工作人员 长春 沈阳 雨水 一路 伴随 国奥队 邪 走 雨 长春 几次 训练 都 大雨 搅和 没想到 沈阳 碰到 事情 一位 国奥 球员 雨水 青睐 不解',
 0)

### 2.2 分词和编码

In [4]:

tokenizer = BertTokenizer.from_pretrained("/kaggle/input/hflchineserobertawwmext")
out = tokenizer.encode(
    text=train_data.__getitem__(0)[0],

    #当句子长度大于max_length时,截断
    truncation=True,
    #一律补pad到max_length长度
    padding='max_length',
    add_special_tokens=True,
    max_length=512,
    return_tensors=None,
    is_split_into_words=True
)
print(out)
tokenizer.decode(out)

#获取字典
#token_dict = tokenizer.get_vocab()
#type(token_dict), len(token_dict), '月光' in token_dict
#添加新词
#tokenizer.add_token(new_tokens=['月光','希望'])
#添加新符号
#tokenizer.add_special_tokens({'eos_token':'[EOS]'})

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RobertaTokenizer'. 
The class this function is called from is 'BertTokenizer'.


[101, 7716, 3236, 3195, 2692, 1912, 1358, 839, 1744, 1952, 6356, 2664, 3187, 1937, 1920, 7433, 3419, 1912, 7471, 4712, 3668, 2157, 1092, 6381, 5442, 987, 762, 7433, 3755, 7345, 2845, 6887, 3341, 1168, 3755, 7345, 1744, 1952, 7339, 898, 4197, 3030, 5564, 7433, 3717, 1737, 2817, 3299, 3189, 678, 1286, 1744, 1952, 7339, 3189, 2382, 6378, 5298, 1086, 2428, 1920, 7433, 2397, 2817, 3187, 1937, 722, 678, 7339, 1447, 2714, 6651, 1146, 7164, 5770, 5770, 3119, 1767, 3189, 677, 1286, 1744, 1952, 7339, 1952, 860, 704, 2552, 1912, 1767, 6378, 5298, 7346, 3756, 3756, 3698, 6496, 7564, 2845, 3227, 4850, 2496, 1921, 678, 1286, 3755, 7345, 1920, 7433, 2401, 1962, 7339, 824, 677, 1286, 6378, 5298, 2397, 2817, 678, 1286, 4157, 2496, 4413, 7339, 2850, 6809, 6378, 5298, 1767, 1920, 7433, 1126, 702, 2207, 3198, 692, 3690, 977, 678, 3341, 2849, 6407, 671, 6407, 2578, 2428, 4413, 7339, 2496, 1921, 678, 1286, 891, 6121, 6378, 5298, 1146, 7164, 1921, 3698, 6760, 1962, 6839, 6496, 924, 2844, 4413, 1447, 1744, 19

'[CLS] 马 晓 旭 意 外 受 伤 国 奥 警 惕 无 奈 大 雨 格 外 青 睐 殷 家 军 记 者 傅 亚 雨 沈 阳 报 道 来 到 沈 阳 国 奥 队 依 然 摆 脱 雨 水 困 扰 月 日 下 午 国 奥 队 日 常 训 练 再 度 大 雨 干 扰 无 奈 之 下 队 员 慢 跑 分 钟 草 草 收 场 日 上 午 国 奥 队 奥 体 中 心 外 场 训 练 阴 沉 沉 气 象 预 报 显 示 当 天 下 午 沈 阳 大 雨 幸 好 队 伍 上 午 训 练 干 扰 下 午 点 当 球 队 抵 达 训 练 场 大 雨 几 个 小 时 丝 毫 停 下 来 抱 试 一 试 态 度 球 队 当 天 下 午 例 行 训 练 分 钟 天 气 转 好 迹 象 保 护 球 员 国 奥 队 中 止 当 天 训 练 全 队 返 回 酒 店 雨 训 练 足 球 队 稀 罕 奥 运 会 即 将 全 队 变 得 娇 贵 沈 阳 一 周 训 练 国 奥 队 保 证 现 有 球 员 不 再 出 现 意 外 伤 病 情 况 影 响 正 式 比 赛 这 一 阶 段 控 制 训 练 受 伤 控 制 感 冒 疾 病 队 伍 放 在 位 置 抵 达 沈 阳 后 卫 冯 萧 霆 训 练 冯 萧 霆 月 日 长 春 患 上 感 冒 参 加 日 塞 尔 维 亚 热 身 赛 队 伍 介 绍 冯 萧 霆 发 烧 症 状 两 天 静 养 休 息 感 冒 恢 复 训 练 冯 萧 霆 例 子 国 奥 队 对 雨 中 训 练 显 得 谨 慎 担 心 球 员 受 凉 引 发 感 冒 非 战 斗 减 员 女 足 队 员 马 晓 旭 热 身 赛 受 伤 导 致 无 缘 奥 运 前 科 沈 阳 国 奥 队 格 外 警 惕 训 练 嘱 咐 队 员 动 作 再 出 事 情 一 位 工 作 人 员 长 春 沈 阳 雨 水 一 路 伴 随 国 奥 队 邪 走 雨 长 春 几 次 训 练 都 大 雨 搅 和 没 想 到 沈 阳 碰 到 事 情 一 位 国 奥 球 员 雨 水 青 睐 不 解 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]

### 2.3 定义批处理函数

In [5]:
def collate_fn(data):
    sents = [i[0] for i in data]
    labels = [i[1] for i in data]
  #编码
    data = tokenizer.batch_encode_plus(batch_text_or_text_pairs=sents,
                                 truncation=True,
                                 padding='max_length',
                                 max_length=512,
                                 return_tensors='pt',
                                 return_length=True,
                                       is_split_into_words=True,)
  #input_ids:编码之后的数字
  #attention_mask:补零的位置三0，其他位置是1
    input_ids = data['input_ids'].to(device)
    attention_mask = data['attention_mask'].to(device)
    token_type_ids = data['token_type_ids'].to(device)
    labels = torch.LongTensor(labels).to(device)

    return input_ids, attention_mask, token_type_ids, labels


#导入数据，这一步合并到trainner中了

train_loader = DataLoader(dataset=train_data,
                    batch_size=TRAIN_BATCH_SIZE,
                    collate_fn=collate_fn,
                    shuffle=True,
                    drop_last=True)

val_loader = DataLoader(dataset=val_data,
                           batch_size=VAL_BATCH_SIZE,
                           collate_fn=collate_fn,
                           shuffle=True,
                           drop_last=True)

test_loader = DataLoader(dataset=test_data,
                           batch_size=TEST_BATCH_SIZE,
                           collate_fn=collate_fn,
                           shuffle=True,
                           drop_last=True)

# for i, (input_ids, attention_mask, token_type_ids, labels) in enumerate(train_loader):
#     break
# print(len(train_loader))
# input_ids.shape, attention_mask.shape, token_type_ids.shape, labels.shape


# for batch in train_dataloader:
#   batch = {k: v.to(device) for k, v in batch.items()}
#   outputs = model(**batch)

## 加载预训练模型

In [6]:
#加载预训练模型
#CN_roberta_wwm_ext = BertModel.from_pretrained("/kaggle/input/hflchineserobertawwmext").to(device)
# #不使用finetune时直接冻结预训练模型的参数
# for param in wobert_pretrained.parameters():
#     param.requires_grad_(False)

#模型试算
#out = bert_bc_pretrained(input_ids=input_ids,
#                 attention_mask=attention_mask,
#                 token_type_ids=token_type_ids)

#out.last_hidden_state.shape

#模型参数查看
#print('param_num: ' + str(sum([i.nelement() for i in CN_roberta_wwm_ext.parameters()]) / 10000))

## 定义下游任务模型

In [7]:
# class Downstream_Model(nn.Module):
#     def __init__(self, ):
#         super().__init__()
#         self.fc = nn.Linear(768,10)
  
#     def forward(self, input_ids, attention_mask, token_type_ids):
#         out = CN_roberta_wwm_ext(input_ids=input_ids,
#                  attention_mask=attention_mask,
#                  token_type_ids=token_type_ids)
      
#         out = self.fc(out.last_hidden_state[:, 0])
#         out = out.softmax(dim=1)

#         return out


class Classifier(nn.Module):
    # 加个全连接 进行分类
    def __init__(self, num_cls):
        super(Classifier, self).__init__()
        self.dense1 = torch.nn.Linear(768, 192)
        self.dense2 = torch.nn.Linear(192, num_cls)
        self.activation = torch.nn.Tanh()
        self.dropout = torch.nn.Dropout(0.1)

    def forward(self, x):
        x = self.dropout(x)
        x = self.dense1(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.dense2(x)
        return x


class RoBerta_CLS_Model(nn.Module):
    def __init__(self, label_num):
        super(RoBerta_CLS_Model, self).__init__()
        self.config = BertConfig.from_pretrained('/kaggle/input/hflchineserobertawwmext/config.json')
        self.config.output_hidden_states = True   # 输出所有的隐层
        self.config.output_attentions = True  # 输出所有注意力层计算结果
        self.roberta = BertModel.from_pretrained('/kaggle/input/hflchineserobertawwmext', config=self.config)

        num_cls = label_num
        # self.highway = Highway(size=768, num_layers=3)
        self.classifier = Classifier(num_cls)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output = self.roberta(input_ids=input_ids, 
                              attention_mask=attention_mask, 
                              token_type_ids=token_type_ids)
        # output[0].size(): batch_size, max_len, hidden_size
        # output[1].size(): batch_size, hidden_size
        # len(output[2]): 13, 其中一个元素的尺寸: batch_size, max_len, hidden_size
        # len(output[3]): 12, 其中一个元素的尺寸: batch_size, 12层, max_len, max_len

        cls_output = output[1]
        # hw_output = self.highway(cls_output)
        logits = self.classifier(cls_output)
        return logits

model = RoBerta_CLS_Model(label_num=10).to(device)
print('param_num: ' + str(sum([i.nelement() for i in model.parameters()]) / 10000))

You are using a model of type roberta to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at /kaggle/input/hflchineserobertawwmext were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassi

param_num: 10241.7226


In [8]:
# for i, (input_ids, attention_mask, token_type_ids, labels) in enumerate(train_loader):
#     break

# with torch.no_grad(): 
#     out = model(input_ids=input_ids,
#                  attention_mask=attention_mask,
#                  token_type_ids=token_type_ids)

# out

## 4. 训练下游任务模型

In [9]:
def train_func(model, output_model_dir, loss_fn, n_epochs, train_loader,scheduler ,var_loader=None):
    for t in range(n_epochs):
        print(f"Epoch {t+1}\n-------------------------------")

        model.train()
        size = len(train_loader.dataset)
        for batch, (input_ids, attention_mask, token_type_ids, labels) in enumerate(train_loader):
            out = model(input_ids=input_ids,
                  attention_mask=attention_mask,
                  token_type_ids=token_type_ids)

            loss = loss_fn(out, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            #以batch为最小检查单位，进行学习率衰减，patience=500
            scheduler.step()

            if batch % 250 == 0:
                loss, current = loss.item(), (batch+1) * len(input_ids)
                print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
        if var_loader != None:
            test_func(dataloader=val_loader, model=model, loss_fn=loss_fn, tp = 'validation')
            
#         if t==round(n_epochS/2) or t==n_epochs-1:
#             output_model_file = os.path.join(output_model_dir, "finetuned_model_epoch_{}.bin".format(t+1))
#             torch.save(model_to_save.state_dict(), output_model_file)
#             print("\n model_saved")
    print("\n training_finished")
    output_model_file = os.path.join(output_model_dir, "finetuned_model_epoch_{}.bin".format(n_epochs))
    torch.save(model.state_dict(), output_model_file)
    
    

def test_func(dataloader, model, loss_fn, tp='validation'):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad(): #停止梯度计算
        for i, (input_ids, attention_mask, token_type_ids, labels) in enumerate(dataloader):
            pred = model(input_ids=input_ids,
                         attention_mask=attention_mask,
                         token_type_ids=token_type_ids)
            test_loss += loss_fn(pred, labels).item()
            correct += (pred.argmax(1) == labels).type(torch.float).sum().item()
  
        test_loss /= num_batches
        correct /= size

    if tp == 'test':
        print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    elif tp == 'validation':
        print(f"Val Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [10]:
import pathlib
pathlib.Path('/kaggle/working/output_model').mkdir(parents=True, exist_ok=True)

In [ ]:
INITIAL_LR = 0.01
optimizer = SGD(model.parameters(),lr = INITIAL_LR)
#scheduler = lr_scheduler.StepLR(optimizer,step_size=5,gamma = 0.5)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.2, patience=500, verbose=True, threshold=0.0001, threshold_mode='rel', cooldown=1, min_lr=1e-8, eps=1e-8)

train_func(train_loader = train_loader,
           var_loader= val_loader, 
           model = model, 
           output_model_dir = '/kaggle/working/output_model',
           loss_fn = nn.CrossEntropyLoss(), 
           optimizer = SGD(model.parameters(), lr=0.01), 
           n_epochs=10)

Epoch 1
-------------------------------
loss: 2.194166  [   20/50000]
loss: 0.308747  [ 4020/50000]
loss: 0.062948  [ 8020/50000]
loss: 0.082081  [12020/50000]
loss: 0.030117  [16020/50000]
loss: 0.327527  [20020/50000]
loss: 0.061981  [24020/50000]
loss: 0.014001  [28020/50000]
loss: 0.016874  [32020/50000]


In [ ]:
# #定义优化器、损失函数、评价指标
# optimizer = AdamW(model.parameters(), lr=5e-4)
# loss_fn = nn.CrossEntropyLoss()
# metric = evaluate.load('accuracy')

# #初始化训练参数
# args = TrainingArguments(output_dir=None,
#                          overwrite_output_dir = False,
#                          evaluation_strategy='epoch',
#                          num_train_epochs = 20,
#                          learning_rate = 1e-4, #优化器默认为AdamW
#                          adam_beta1 = 0.9,
#                          adam_beta2 = 0.999,
#                          adam_epsilon = 1e-8,
#                          weight_decay = 1e-2, #各层的权重衰减
#                          max_grad_norm = 1.0, #梯度裁剪
#                          per_device_eval_batch_size = 128,
#                          per_device_train_batch_size = 128,
#                          lr_scheduler_type = 'linear',
#                          save_strategy = 'epoch',
#                          no_cuda = False,
#                          seed = 1024,
#                          data_seed = 1024,
#                          load_best_model_at_end = False,
#                          metric_for_best_model = 'loss',
#                          greater_is_better = False
#                          )

# class CustomTrainer(Trainer): #可能需要自己继承trainer,覆写一些函数
#     def get_train_dataloader():
#         return
#     def get_test_dataloader():
#         return

# #初始化训练器                         
# trainer = Trainer(
#     model = model,
#     args = args,
#     data_collator = collate_fn, #构建batch
#     train_dataset = train_data,
#     eval_dataset = val_data,
#     compute_metrics = metric,
#     tokenizer = tokenizer
#     #callbacks = 
#     #optimizers = 
# )

## 5. 测试模型效果

In [ ]:
test_func(dataloader=test_loader, 
          model=model, 
          loss_fn = nn.CrossEntropyLoss(), 
          tp='test')

## 6.保存模型

In [ ]:
model.state_dict()